# DROSSEL SCHWABL Forest Fire Model

This simulation is based on the work of:

Drossel, B. and Schwabl, F. (1992), "Self-organized critical forest-fire model." Phys. Rev. Lett. 69, 1629–1632.

In [ ]:
%pylab inline

In [ ]:
import numpy as np
import collections as col
from matplotlib import colors
from scipy.ndimage import measurements
from scipy import ndimage
import os
import pandas as pd

In [ ]:
#if not present, it creates the directory to save gif images
if os.path.isdir("./images_DROSSEL_gif") == False:
    ! mkdir images_DROSSEL_gif

## This function initializes the forest

In [ ]:
def forest_init(p,L):
    
    """
    This function initializes the forest with the following rule:
    
    p = an empty space fills with a tree with probability p
    L = linear dimension of the forest (L,L)
    
    it returns a 2D numpy array with shape (L,L) filled with binary values: 
    0 = empty cell (ground)
    1 = tree
    
    the returned array is padded with zeros 
    
    e.g. forest_init(0.30,50)"""
    
    if (p>1)|(p<0):
        
        return 'p must be a probability in [0,1]'
    
    else:
        r = rand(L-2,L-2)
        foresta = np.zeros((L-2,L-2), dtype = int)
        
        #an empty space fills with a tree with probability p
        foresta[r<p] = 1 
                    
    return np.pad(foresta, pad_width=[(1,1),(1,1)], mode='constant', constant_values=0)

## This function spreads the fire to all forest clusters with at least one burning tree

In [ ]:
def cluster_fire(foresta):
    
    """
    This function takes a forest as input parameter, finds the forest clusters and set on fire
    those with one burning tree. It return the area of the burning forest-clusers and the updated forest.
    """

    foresta_buf = zeros(foresta.size,dtype=int)
    clusters_pos = []
    clusters = []
    
    lbl, nlbl = ndimage.label(foresta)
    lbls = np.arange(1, nlbl+1)

    positions = numpy.arange(foresta.size).reshape(foresta.shape)

    for i in lbls:
        clusters.append(foresta[lbl==i])
        clusters_pos.append(positions[lbl==i])

    for cluster in clusters:                     #it iterates the list of forest clusters
    
        if len(cluster[cluster==2])>0:           #if one cluster tree it's struck by lightning the whole cluster
            cluster.fill(2)                      #it's burned down
    
    for clust, pos in zip(clusters,clusters_pos):

        for i in range(0,len(clust)):
            foresta_buf[pos[i]] = clust[i]
            
    foresta_buf = foresta_buf.reshape(foresta.shape)
    
    only_burning_clusters = foresta_buf.copy()
    only_burning_clusters[only_burning_clusters!=2]=0
    only_burning_clusters[only_burning_clusters==2]=1
    cluster_label, n_cluster = measurements.label(only_burning_clusters)
    areas = measurements.sum(only_burning_clusters, cluster_label, index=arange(cluster_label.max() + 1))
    

    return areas[1:], foresta_buf

## This function burns to the ground all burning forest clusters

In [ ]:
def burning_to_empty(foresta):
    "This function burns down the whole forest cluster if one of its trees it's struck with lightning"
    #The burning cells turn into empty cells
    foresta[foresta==2]=0
    return foresta

## This function update the forest (lightnings and growing trees)

In [ ]:
def tree_grow_and_lightnings(foresta,p,f):
    
    """
    This function takes a forest element (np.array LxL), 
    the probability p of a tree growing in an empy cell, the lightning probability f 
    and returns the updated forest     
    """
    
    if (p>1)|(p<0):
        return 'p must be a probability in [0,1]'
    elif (f>1)|(f<0):
        return 'f must be a probability in [0,1]'
    else:
        #the tree dynamic is confined to the inner forest
        r = rand(foresta.shape[0]-2,foresta.shape[1]-2)
        inner_forest = foresta[1:foresta.shape[0]-1,1:foresta.shape[1]-1]
        inner_forest[(inner_forest==1)&(r<f)] = 2
        inner_forest[(inner_forest==0)&(r<p)] = 1
        foresta[1:foresta.shape[0]-1,1:foresta.shape[1]-1] = inner_forest

    return foresta

## Simulation

In [ ]:
########################### SIMULATION PARAMETERS

L = 256
p_init = 0.30
p_grow = 0.057
f = 0.005
tmax = 700 # each simulation step t consists of 3 different temporal steps 
forest_fire_areas_list = []
cmap1 = colors.ListedColormap(['#4D0000','#228B22','red'])
cmap2 = colors.ListedColormap(['#4D0000','#228B22'])


########################### FOREST INIT

foresta = forest_init(p_init,L) 

########################### SIMULATION DYNAMIC

i = 0    
for t in range(1,tmax):
    #### check if there is any tree struck by lightning and set the colormap
    if len(foresta[foresta ==2]) > 0:
        cmap = cmap1
    else:
        cmap = cmap2

    #### all the cluster with at least one buring tree are set on fire    
    forest_fire_area_t, foresta = cluster_fire(foresta)
    forest_fire_areas_list.append(forest_fire_area_t)
    i+=1
    imshow(foresta,cmap=cmap)
    savefig('./images_DROSSEL_gif/forestFire_snapshot_time_'+str(i)+'.png')

    
    #### all the buring cluster are burned to the ground  
    burning_to_empty(foresta)
    if len(foresta[foresta ==2]) > 0:
        cmap = cmap1
    else:
        cmap = cmap2
    i+=1
    imshow(foresta,cmap=cmap)
    savefig('./images_DROSSEL_gif/forestFire_snapshot_time_'+str(i)+'.png')

    
    #### some trees are struck by lightnings and other trees grow in empty cells
    tree_grow_and_lightnings(foresta,p_grow,f)
    if len(foresta[foresta ==2]) > 0:
        cmap = cmap1
    else:
        cmap = cmap2
    i+=1
    imshow(foresta,cmap=cmap)
    savefig('./images_DROSSEL_gif/forestFire_snapshot_time_'+str(i)+'.png')
    
num_img = i

In [ ]:
from images2gif import writeGif
from PIL import Image
import os

In [ ]:
immagini = [Image.open('./images_DROSSEL_gif/forestFire_snapshot_time_'+str(i)+'.png') for i in range(1,400)]

In [ ]:
filename = "my_gif_p="+str(p_grow)+"_f="+str(f)+"_DRO.GIF"
writeGif(filename, immagini, duration=0.7)

# Testing if the fire extention behaves according to a power law

$p(x) = x^{-\alpha}$ 

In [ ]:
num_cluster_areaA = col.defaultdict(int)

for clusters_size in forest_fire_areas_list[1:]:
    for cluster_size in clusters_size:
        num_cluster_areaA[int(cluster_size)] += 1

In [ ]:
# to save simulation results
area_list = []
p_area_list = []

for key, value in num_cluster_areaA.iteritems():
    area_list.append(key)
    p_area_list.append(value)

fout=open('simulation_areas_freq.csv','w')  
fout.write('area, area_freq\n')
for i in range(0,len(area_list)):
    fout.write(str(area_list[i])+','+str(p_area_list[i])+'\n')
fout.close()

In [ ]:
#if you want to analyze an old simulation
area_list = []
p_area_list = []

fin = open('simulation_areas_freq.csv','r')
l = fin.readlines()

for line in l[1:]:
    s = line.strip(' ').split(',')  

    area_list.append(int(s[0]))
    p_area_list.append(int(s[1]))
 

fin.close()    

In [ ]:
ax = plt.gca()
ax.scatter(area_list,p_area_list)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('$log_{10}(fire \ area\ A)$')
ax.set_ylabel('$log_{10}(frequency\ of\ fire \ area \ f_A)$')
ax.set_xlim(right=max(area_list))
ax.set_ylim(bottom=min(p_area_list))
#ax.axvline(x=256,color='r')
savefig('./powerlaw_distribution.png')

# Analysis using powerlaw library

http://www.plosone.org/article/info%3Adoi%2F10.1371%2Fjournal.pone.0085777

In [ ]:
import powerlaw

A fit of a data set to various probability distributions, namely power laws. 

For fits to power laws, the methods of Clauset et al. 2007 are used.

These methods identify the portion of the tail of the distribution that follows a power law, beyond a value xmin. If no xmin is provided, the optimal one is calculated and assigned at initialization.

In [ ]:
fit = powerlaw.Fit(p_area_list,xmin=1,xmax=256)

The goodness of these distribution fits can be compared with **distribution_compare** method. 

The test is based on the **likelihood ratio**, which expresses how many times more likely the data are under one model than the other. This likelihood ratio, or equivalently its logarithm, can then be used to compute a p-value.

Among the supported distributions is the exponentially truncated power law, which has the power law's scaling behavior over some range but is truncated by an exponentially bounded tail.

A power law with an exponential cutoff is simply a power law multiplied by an exponential function:
    $f(x) \propto x^{\alpha}e^{\lambda x}$

In [ ]:
R,p=fit.distribution_compare('power_law', 'truncated_power_law')
print "R = ",R
print "p = ",p

R is the loglikelihood ratio between the two candidate distributions (how many times more likely the data are under one model than the other).

p is the p-value (the probability of obtaining a test statistic result at least as extreme or as close to the one that was actually observed, assuming that the null hypothesis is true)

This number will be positive if the data is more likely in the first distribution, and negative if the data is more likely in the second distribution. 

In [ ]:
R, p = fit.distribution_compare('exponential', 'truncated_power_law')
print "R = ",R
print "p = ",p

In [ ]:
print str(fit.truncated_power_law.parameter1_name)+': '+ str(fit.truncated_power_law.parameter1)
print str(fit.truncated_power_law.parameter2_name)+': '+ str(fit.truncated_power_law.parameter2)